In [ ]:
import torch
import torch.nn as nn
from torchvision.models import inception_v3, Inception_V3_Weights
import td_load_data_inception
import td_run_model2

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_classes = 3  # Adjust this to match the number of classes (e.g., high, medium, low)
num_epochs = 20
batch_size = 16
learning_rate = 0.005

In [4]:
class InceptionV3MultiClassClassifier(nn.Module):
    def __init__(self, num_classes):
        super(InceptionV3MultiClassClassifier, self).__init__()
        self.base_model = inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)

        # Do not modify the original fully connected (fc) layer
        self.base_model.aux_logits = False  

        self.new_fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), 
            nn.Flatten(), 
            nn.Linear(self.base_model.fc.out_features, num_classes),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.base_model(x) 
        return x

In [7]:

if __name__ == "__main__":
    # Model initialization
    model = InceptionV3MultiClassClassifier(num_classes=num_classes).to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    

    # Load data
    train_loader, validation_loader, test_loader, classes = td_load_data_inception.create_data(batch_size=batch_size)
    
    # Train and validate the model
    td_run_model2.train(num_epochs, device, model, criterion, optimizer, train_loader, validation_loader, num_classes)
   
    # Test the model
    td_run_model2.test(device, model, test_loader, num_classes)

Epoch [1/20], Loss: 1.7725
Validation Accuracy: 81.88% Macro-averaged Precision: 0.8091, Macro-averaged Recall: 0.8054
ROC AUC scores for each class: {'Class 0': 0.958, 'Class 1': 0.9648, 'Class 2': 0.8923}
Epoch [2/20], Loss: 0.5030
Validation Accuracy: 84.70% Macro-averaged Precision: 0.8380, Macro-averaged Recall: 0.8345
ROC AUC scores for each class: {'Class 0': 0.9727, 'Class 1': 0.977, 'Class 2': 0.9261}
Epoch [3/20], Loss: 0.4152
Validation Accuracy: 86.63% Macro-averaged Precision: 0.8616, Macro-averaged Recall: 0.8553
ROC AUC scores for each class: {'Class 0': 0.9803, 'Class 1': 0.9783, 'Class 2': 0.9341}
Epoch [4/20], Loss: 0.3330
Validation Accuracy: 87.76% Macro-averaged Precision: 0.8706, Macro-averaged Recall: 0.8678
ROC AUC scores for each class: {'Class 0': 0.9834, 'Class 1': 0.982, 'Class 2': 0.9402}
Epoch [5/20], Loss: 0.2954
Validation Accuracy: 88.33% Macro-averaged Precision: 0.8790, Macro-averaged Recall: 0.8727
ROC AUC scores for each class: {'Class 0': 0.9851, '

In [ ]:
# Define the model architecture with the same number of classes used during training
model = InceptionV3MultiClassClassifier(num_classes=num_classes).to(device)

# Load the state dict
# state_dict = torch.load('weights/31.pth')
state_dict = torch.load('weights/31.pth', map_location=torch.device('cpu'))     # Running locally

# Load the state dict into the model
model.load_state_dict(state_dict)

model.eval()  # Set the model to evaluation mode

# Load the test data using the function in td_load_data.py
_, _, test_loader, classes = td_load_data_inception.create_data(batch_size=batch_size)

# Evaluate the model on the test set
td_run_model2.test_model_on_test_data(device, model, test_loader)